<a href="https://colab.research.google.com/github/adeshmukh/gaiip/blob/main/notebooks/Intro_to_Agentic_Architectures_BabyAGI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BabyAGI

This notebook contains an illustrative implementation of the BabyAGI architecture.

<a target="_blank" href="https://colab.research.google.com/github/https://colab.research.google.com/github/adeshmukh/gaiip/blob/main/notebooks/Intro_to_Agentic_Architectures_BabyAGI.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Setup

1. Install dependencies
2. Set up API keys

In [1]:
%%capture

!pip install langchain
!pip install langchain-community
!pip install langgraph
!pip install langchainhub

!pip install langchain_openai

!pip install faiss-cpu
!pip install openai

In [2]:
from google.colab import userdata
import os


key_names = [
  'OPENAI_API_KEY',
]

for key_name in key_names:
  os.environ[key_name] = userdata.get(key_name)

## BabyAGI implementation

In [20]:
import os
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore


class BabyAGI:
  def __init__(self, task, llm, embeddings, max_iterations=5):
    self.task = task
    self.llm = llm
    self.embeddings = embeddings
    self.max_iterations = max_iterations
    self.vectorstore = None

  def run(self):
    task_list = [self.task]
    for _ in range(self.max_iterations):
      if not task_list:
        break
      current_task = task_list.pop(0)
      current_task = current_task.replace('\\n', ' ')
      result = self.execute_task(current_task)
      self.add_to_vectorstore(result)
      new_tasks = self.create_new_tasks(result)
      task_list.extend(new_tasks)
      print(f"\n\n--- COMPLETED TASK: \n{current_task}")
      print(f"\n\n=== RESULT: \n{result}")
      new_tasks_display = '\n'.join(new_tasks)
      print(f"\n\n+++ NEW TASKS: \n{new_tasks_display}")

  def execute_task(self, task):
    prompt = PromptTemplate.from_template("Complete the following task: {task}")
    chain = prompt | self.llm
    response = chain.invoke({"task": task})
    return response.content  # Extract the content from the AIMessage

  def create_new_tasks(self, result):
    prompt = PromptTemplate.from_template("Based on this result, suggest up to 3 new tasks: {result}")
    chain = prompt | self.llm | CommaSeparatedListOutputParser()
    response = chain.invoke({"result": result})
    return response  # This should already be a list due to the CommaSeparatedListOutputParser

  def add_to_vectorstore(self, text):
    if self.vectorstore is None:
      self.vectorstore = FAISS.from_texts([text], self.embeddings)
    else:
      self.vectorstore.add_texts([text])

In [22]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings


def run_baby_agi(task):
  llm = ChatOpenAI(model="gpt-4o", temperature=0,)
  embeddings = OpenAIEmbeddings()
  baby_agi = BabyAGI(task, llm, embeddings)
  baby_agi.run()

## Usage

In [23]:
run_baby_agi("Develop a marketing strategy for a new smartphone")



--- COMPLETED TASK: 
Develop a marketing strategy for a new smartphone


=== RESULT: 
### Marketing Strategy for a New Smartphone

#### 1. **Market Research and Analysis**
   - **Target Audience Identification**: Identify the primary and secondary target audiences. This could include tech enthusiasts, professionals, students, and budget-conscious consumers.
   - **Competitive Analysis**: Analyze competitors' strengths and weaknesses. Identify gaps in the market that your smartphone can fill.
   - **Consumer Insights**: Conduct surveys, focus groups, and social media listening to understand consumer needs, preferences, and pain points.

#### 2. **Unique Selling Proposition (USP)**
   - **Innovative Features**: Highlight unique features such as advanced camera technology, superior battery life, cutting-edge design, or exclusive software capabilities.
   - **Value Proposition**: Emphasize the value for money, durability, and after-sales service.

#### 3. **Brand Positioning**
   - **Bra